## Les méthodes de délimitation du programme ZuPer

#### La méthode standard

In [1]:
def standard_method(self, PI):
    # take contributions values
    c = self.dict_results['contributions'].flatten()
    
    # quicksort algorithm used to sort contributions values
    index = np.argsort(c)[::-1]
    
    # create the zu zone variable
    z = np.zeros((self.nnodes,))
    
    # define the res variable to stock results of contributions
    res = 0
    
    # define the limits of contributions and the tau variable
    self.PI_LIM = PI*self.qw
    self.tau=0
    
    # for loop to determine which nodes are usefull to determine the Zu zone
    for i in index:
        res+=c[i]
        z[i] = 1
        self.tau+=1
        if res>=self.PI_LIM:
            break;
    
    z.shape = (self.nby_nd, self.nbx_nd)
    self.dict_results['zu'] = z

#### La méthode meanlife

In [2]:
def meanlife_method(self, PI):
    # take capture probabilities, exit-time and contributions values
    p, e, c = self.dict_results['probability'].flatten(), self.dict_results['exit-time'].flatten(), self.dict_results['contributions'].flatten()
    
    # create the zu zone variable
    z = np.zeros((self.nnodes,))
    
    # quicksort algorithm used to sort exit-time values
    ind_e = np.argsort(e)
    
    # define the res variable to stock results of contributions
    res = 0
    
    # define the limits of contributions
    self.PI_LIM = PI * self.qw

    # instance the results and tau variable
    res+=c[self.id_well]
    z[self.id_well] = 1
    self.tau=1

    # for loop to determine which nodes are usefull to determine the Zu zone
    for i in ind_e:
        if e[i]!=0 and p[i]>0.01:
            res+=c[i]
            z[i] = 1
            self.tau+=1
            if res>=self.PI_LIM:
                break;

    # calculate the differents coefficients of the Zu zone
    z.shape = (self.nby_nd, self.nbx_nd)
    self.dict_results['zu'] = z

#### La méthode basique ou CFR

In [3]:
def cfr_method(self, PI):
    # determine the radius
    r = np.sqrt( (self.qw*PI) / (np.pi*self.r_mean) )
    
    # determine the zu zone by the circle formula with the previous calculated radius
    d = np.sqrt((self.x - self.xw)**2 + (self.y - self.yw)**2) < r
    
    self.tau = np.unique(d, return_counts=True)[1][1]
    self.a_zu = round(self.tau*self.s/10000,2)
    self.rho = round(self.tau/self.nnodes * 100,2)
    
    # switch boolean value into integers values for the zu zone
    d = d.astype(float)
    
    # resize the d variable 
    d.resize(self.nby_nd, self.nbx_nd)
    
    # put the zu zone in the results dictionnary
    self.dict_results['zu'] = d

#### La méthode de Lerner

In [4]:
def lerner_method(self, PI):
    # Initiate all parameters that Lerne's solution need to work :
    # R=mean recharge ; b=thickness of aquifer ; T=transmissivity ; n=porosity
    # Q=discharge of well ; xw and yw=position of well in the matrix mesh
    # xmin, xmax, ymin, ymax=minimum and maximum of x-axis and y-axis respectively
    # a = the length of the aquifer ; nx, ny=number of nodes of x-axis and y-axis in the mesh
    R = self.r_mean
    b = self.b
    T = self.k_mean
    poro = self.poro
    Q = PI*self.qw 
    xw, yw = self.xw, self.yw
    xmin, xmax = self.xmin, self.xmax
    ymin, ymax = self.ymin, self.ymax
    a = (ymax - ymin) 
    nx = self.nbx_nd
    ny = self.nby_nd
    
    # define axis vector in the mesh
    xv = np.linspace(xmin,xmax,nx)
    yv = np.linspace(ymin,ymax,ny)
    xx, yy = np.meshgrid(xv, yv)
    
    # define the parameter for the size of calculation
    xl = ymax - yy
    yl = xx 
    xlw = ymax - yw
    ylw = xw
    
    # define the initial velocity in x-axis and y-axis for calculation
    vxlr = R * xl / (b * poro)
    vylr = 0
    
    # define the limit of y
    ym = np.pi * (yl - ylw) / (2 * a)
    
    # function of calculation of velocity in x-axis
    def Ux(xp):
        xm = np.pi * (xl - xp) / (2 * a)
        xp = np.pi * (xl + xp) / (2 * a)
    
        return - Q / (8 * a * b * poro) * ( np.sin(xm) / (np.cosh(ym) - np.cos(xm)) +
                                    np.sin(xp) / (np.cosh(ym) + np.cos(xp)))
    
    # function of calculation of velocity in x-axis
    def Uy(xp):
        xm = np.pi * (xl - xp) / (2 * a)
        xp = np.pi * (xl + xp) / (2 * a)
    
        return   - Q * np.sinh(ym) / (8 * a * b * poro) * ( 1 / (np.cosh(ym) - np.cos(xm)) +
                                                1 / (np.cosh(ym) + np.cos(xp))) 
    
    # use of well image to calculate velocity from the well
    vxlw = Ux(xlw) + Ux(-xlw)
    vylw = Uy(xlw) + Uy(-xlw)
    
    vxl = vxlr + vxlw
    vyl = vylr + vylw
    
    vx = vyl
    vy = - vxl
    
    # function use to define the equation needed to be resolve
    def f(x):
        xm = np.pi * (x - xlw) / ( a)
        xp = np.pi * (x + xlw) / ( a)
        return R * x / (b * poro) - Q / (2 * a * b * poro) * (1 / np.sin(xm) - 1 / np.sin(xp))
    
    # parameter of differential used in the resolution of the differential equation
    eps = 0.1
    res = root_scalar(f, bracket=[xlw + eps, a])
    xls = res.root
    xs = xw
    ys = ymax - xls

    # function to determine the velocity field
    def vxy(t, pos):
        # calculate initial variables (position, mesh) 
        x, y = pos
        xl = ymax - y
        yl = x
        xlw = ymax - yw
        ylw = xw
        ym = np.pi * (yl - ylw) / (2 * a)

       # return the velocity field for the x-axis (Ux) and y-axis (Uy)
        def Ux(xk):
            xm = np.pi * (xl - xk) / (2 * a)
            xp = np.pi * (xl + xk) / (2 * a)
            return - Q / (8 * a * b * poro) * ( np.sin(xm) / (np.cosh(ym) - np.cos(xm)) +
                                    np.sin(xp) / (np.cosh(ym) + np.cos(xp)))
    
        def Uy(xk):
            xm = np.pi * (xl - xk) / (2 * a)
            xp = np.pi * (xl + xk) / (2 * a)
            return - Q * np.sinh(ym) / (8 * a * b * poro) * ( 1 / (np.cosh(ym) - np.cos(xm)) +
                                                1 / (np.cosh(ym) + np.cos(xp))) 

        # use image well theory to determine the true velocity field
        vx = Uy(xlw) + Uy(-xlw)
        vy = - Ux(xlw) - Ux(-xlw)  - R * xl / (b * poro)

        # return the results
        return [-vx, -vy]
    
    # resolve the differential equation with Newton-Ralphson method
    sol = solve_ivp(vxy, [0, 1e6], [xs + 30*eps, ys], max_step = 50)
    
    # get values of resolution
    ybound = sol.y[1] 
    xrightbound = sol.y[0]
    xleftbound = 2 * xw - xrightbound
    
    # define a mesh and interpolate the function for discretisation method
    cx = np.linspace(xmin, xmax, nx)
    cy = np.linspace(ymin, ymax, ny)
    dx = (xmax-xmin)/nx
    xx, yy = np.meshgrid(cx, cy)
    capture = np.zeros(yy.shape)
    f = interpolate.interp1d(ybound, xleftbound)
    
    # For loop to integrate the Lerner solution with a discretisation
    # method from the bottom to the top of model
    for M in range(ny):
        xcurent = yy[M,0]
        if xcurent < ybound[0]:
            continue
        elif xcurent > ybound[-1]:
            continue
        else:
            ytop = f(xcurent)
            jtop = int( (ytop-xmin)/dx)
            jbot = -jtop
            capture[M, jtop:jbot] = 1
        
    # keep Zu zone in "dict_results" and determine coefficients (efficiency and surface)
    self.dict_results['zu'] = capture[::-1].copy()
    self.dict_results['zu'].shape = (self.nby_nd, self.nbx_nd)
    self.tau = np.unique(capture[::-1].copy(), return_counts=True)[1][1]

#### La méthode de Bear-Jacobs

In [5]:
def bear_jacobs_method(self, PI):
    y=np.linspace(-np.pi,np.pi,100)
    
    def x_bear_jacob(y,t,k=0):
        a=np.cos(y)
        b=np.sin(y)/y
        c=np.exp(-t)
        x=-a/b - sp.special.lambertw(-c/(b*np.exp(a/b)),k=k)
        
        if x.imag==0:
            return x.real
        else:
            return 9999
    
    def bear_jacob(t,n_point=1000):
        Y_0=np.linspace(-np.pi+0.1,np.pi-0.1,n_point)
        Y_1=np.linspace(-np.pi+0.1,np.pi-0.1,n_point)
        X_0=np.zeros(n_point)
        X_1=np.zeros(n_point)
        for i in range(len(Y_0)):
            X_0[i]=x_bear_jacob(Y_0[i],t)
            X_1[i]=x_bear_jacob(Y_1[i],t,k=-1)
        
        X=np.concatenate((X_0,X_1[::-1]))
        Y=np.concatenate((Y_0,Y_1[::-1]))
        
        return X[X!=9999],Y[X!=9999]
    
    def changement_coordonnées(X,Y,xw,yw):
        new_x=-Y+xw
        new_y=X+yw
        
        return new_x,new_y
    
    def full_bear_jacob(t,q0,Qi,n,xw,yw,n_point=1000):
        tb=2*np.pi*(q0**2)*t/n/Qi
        xb,yb=bear_jacob(tb,n_point=n_point)
        x=xb*Qi/2/np.pi/q0
        y=yb*Qi/2/np.pi/q0
        
        return changement_coordonnées(x,y,xw,yw)
    
    def bear_and_jacobs_opti(t,Q,xi,Recharge,gdf_coord):
        return abs(bear_and_jacobs_recharge(t,Q,Recharge,gdf_coord)-Q*xi)
    
    def bear_and_jacobs_recharge(t,Q,Recharge,gdf_coord):
        t=t[0]
        resx,resy=full_bear_jacob(t=t,q0=self.r_mean/86400 * self.s * self.nnodes,Qi=Q,n=0.2,xw=self.xw,yw=self.yw,n_point=1000)
        Poly_zu=Polygon(zip(resx,resy))
    
        Zu=gdf_coord.within(Poly_zu)
        Rf=Recharge.flatten()
        return np.sum(Rf[Zu])
    
    def bear_and_jacobs_ZU(Q,xi,Recharge):
        dd = {'X':self.x.flatten().copy(), 'Y':self.y.flatten().copy()}
        
        dataf = gpd.GeoDataFrame(dd, geometry=gpd.points_from_xy(dd['X'], dd['Y']))
        
        x0=[3e6]
        opti=minimize(bear_and_jacobs_opti,x0,args=(Q,xi,Recharge,dataf), method='Nelder-Mead')
        resx,resy=full_bear_jacob(t=x0[0],q0=self.r_mean /86400 *self.s *self.nnodes/3000, Qi=Q*xi, n=0.2, xw=self.xw, yw=self.yw, n_point=1000)
        Poly_zu=Polygon(zip(resx,resy))
    
        Zu=dataf.within(Poly_zu)
    
        return Zu
    
    # solve the bear-jacobs equation
    Zu = bear_and_jacobs_ZU(self.qw/86400, PI, self.dict_params['rech']/86400)
    zzz = Zu.values
    
    # if condition to determine if we have True values after delimitation
    if True in zzz:
        self.tau = np.unique(zzz, return_counts=True)[1][1]
    
    else:
        self.tau = 0
    
    # Define coefficients (efficiency, surface of Zu zone) and keep Zu matrix in "dict_results"
    zzz = zzz.astype(float)
    zzz.resize(287, 123)
    self.dict_results['zu'] = zzz

## Choisir une méthode spécifique

In [6]:
def choose_method(self, method, PI):
    # Method to choose by "method" argument, and use all 
    # parameters of the method chosen found in "list_params_method".
    if method=='standard':
        self.standard_method(PI)

    elif method=='meanlife':
        self.meanlife_method(PI)

    elif method=='cfr':
        self.cfr_method(PI)
    
    elif method=='lerner':
        self.lerner_method(PI)
        
    elif method=='bear-jacobs':
        self.bear_jacobs_method(PI)